# Detecting Pneumonia from X-ray using CNN(Image recognition)

### Reading the Images data

In [1]:
# Deep Learning CNN model to recognize face

'''########################## IMAGE PRE-PROCESSING for TRAINING and TESTING data ##############################'''

TrainingImagePath='C:/Users/Administrator/Desktop/chest_xray/train'

from keras.preprocessing.image import ImageDataGenerator

# Defining pre-processing transformations on raw images of training data
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True)

# Defining pre-processing transformations on raw images of testing data
test_datagen = ImageDataGenerator(rescale=1./255)

# Generating the Training Data
training_set = train_datagen.flow_from_directory(
        TrainingImagePath,
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')


# Generating the Testing Data
test_set = test_datagen.flow_from_directory(
        TrainingImagePath,
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')

# Printing class labels for each Xray
test_set.class_indices

Found 5216 images belonging to 2 classes.
Found 5216 images belonging to 2 classes.


{'NORMAL': 0, 'PNEUMONIA': 1}

### Creating a list of faces and classes

In [2]:
'''#################### Creating lookup table for all faces ##############################'''
# class_indices have the numeric tag for each face
TrainClasses=training_set.class_indices

# Storing the face and the numeric tag for future reference
ResultMap={}
for faceValue,faceName in zip(TrainClasses.values(),TrainClasses.keys()):
    ResultMap[faceValue]=faceName

# Saving the face map for future reference
import pickle
with open("C:/Users/Administrator/Desktop/IVY/chest_xray/ResultsMap.pkl", 'wb') as f:
    pickle.dump(ResultMap, f, pickle.HIGHEST_PROTOCOL)

print("Mapping of Xray's and its ID",ResultMap)

# The number of neurons for the output layer is equal to the number of faces
OutputNeurons=len(ResultMap)
print('\n The Number of output neurons: ', OutputNeurons)

Mapping of Xray's and its ID {0: 'NORMAL', 1: 'PNEUMONIA'}

 The Number of output neurons:  2


### Creating the CNN model

In [3]:
'''######################## Create CNN deep learning model ####################################'''
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense

'''Initializing the Convolutional Neural Network'''
classifier= Sequential()

''' STEP--1 Convolution
# Adding the first layer of CNN
# we are using the format (64,64,3) because we are using TensorFlow backend
# It means 3 matrix of size (64X64) pixels representing Red, Green and Blue components of pixels
'''
classifier.add(Convolution2D(32, kernel_size=(5, 5), strides=(1, 1), input_shape=(64,64,3), activation='relu'))

'''# STEP--2 MAX Pooling'''
classifier.add(MaxPool2D(pool_size=(2,2)))

'''############## ADDITIONAL LAYER of CONVOLUTION for better accuracy #################'''
classifier.add(Convolution2D(64, kernel_size=(5, 5), strides=(1, 1), activation='relu'))

classifier.add(MaxPool2D(pool_size=(2,2)))


'''# STEP--3 FLattening'''
classifier.add(Flatten())

'''# STEP--4 Fully Connected Neural Network'''
classifier.add(Dense(64, activation='relu'))

classifier.add(Dense(OutputNeurons, activation='softmax'))

'''# Compiling the CNN'''
#classifier.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
classifier.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=["accuracy"])

### Fitting the model

In [4]:
import time
# Measuring the time taken by the model to train
StartTime=time.time()

# Starting the model training
classifier.fit_generator(
                    training_set,
                    steps_per_epoch=30,
                    epochs=10,
                    validation_data=test_set,
                    validation_steps=10)

EndTime=time.time()
print("############### Total Time Taken: ", round((EndTime-StartTime)/60), 'Minutes #############')

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
30/30 [==============================] - 50s 2s/step - loss: 0.5451 - accuracy: 0.7625 - val_loss: 0.3850 - val_accuracy: 0.8625
Epoch 2/10
30/30 [==============================] - 48s 2s/step - loss: 0.3884 - accuracy: 0.8344 - val_loss: 0.2814 - val_accuracy: 0.9125
Epoch 3/10
30/30 [==============================] - 44s 1s/step - loss: 0.2714 - accuracy: 0.8823 - val_loss: 0.2275 - val_accuracy: 0.9000
Epoch 4/10
30/30 [==============================] - 41s 1s/step - loss: 0.2292 - accuracy: 0.9083 - val_loss: 0.1635 - val_accuracy: 0.9469
Epoch 5/10
30/30 [==============================] - 42s 1s/step - loss: 0.2000 - accuracy: 0.9094 - val_loss: 0.1259 - val_accuracy: 0.9594
Epoch 6/10
30/30 [==============================] - 55s 2s/step - loss: 0.2040 - accuracy: 0.9281 - val_loss: 0.1518 - val_accuracy: 0.9406
Epoch 7/10
30/30 [==============================] - 48s 2s/step - loss: 0.1819 - acc

### Saving the model

In [5]:
## Saving the model
classifier.save("C:/Users/Administrator/Desktop/IVY/chest_xray/classifier.pkl")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: C:/Users/Administrator/Desktop/IVY/chest_xray/classifier.pkl\assets


### Testing the model on a different face

In [6]:
'''########################## Making single predictions ############################'''
import numpy as np
from keras.preprocessing import image

testImage='C:/Users/Administrator/Desktop/IVY/chest_xray/test/3.jpg'
test_image=image.load_img(testImage,target_size=(64, 64))
test_image=image.img_to_array(test_image)

test_image=np.expand_dims(test_image,axis=0)

result=classifier.predict(test_image,verbose=0)
#print(training_set.class_indices)

print('####'*10)
print('Prediction is: ',ResultMap[np.argmax(result)])

########################################
Prediction is:  NORMAL
